
Nguyễn Phan Mạnh Hùng - 1312727


In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cPickle
import gzip
# You can also import other things ...
import copy

In [2]:
def read_mnist(mnist_file):
    """
    Reads MNIST data.
    
    Parameters
    ----------
    mnist_file : string
        The name of the MNIST file (e.g., 'mnist.plk.gz').
    
    Returns
    -------
    (X_train, Y_train, X_val, Y_val, X_test, Y_test) : tuple
        X_train : numpy array, shape (N=50000, d+1=785)
            Input vectors of the training set.
        Y_train: numpy array, shape (N=50000)
            Outputs of the training set.
        X_val : numpy array, shape (N=10000, d+1=785)
            Input vectors of the validation set.
        Y_val: numpy array, shape (N=10000)
            Outputs of the validation set.
        X_test : numpy array, shape (N=10000, d+1=785)
            Input vectors of the test set.
        Y_test: numpy array, shape (N=10000)
            Outputs of the test set.
    """
    f = gzip.open(mnist_file, 'rb')
    train_data, val_data, test_data = cPickle.load(f)
    f.close()
    
    X_train, Y_train = train_data
    X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
    
    X_val, Y_val = val_data
    X_val = np.hstack((np.ones((X_val.shape[0], 1)), X_val))
    
    X_test, Y_test = test_data
    X_test = np.hstack((np.ones((X_test.shape[0], 1)), X_test))
    
    return X_train, Y_train, X_val, Y_val, X_test, Y_test

In [3]:
def tanh(Z):
    """
    Computes tanh function for each element of numpy array Z.
    """
    ez = np.exp(Z)
    esz = np.exp(-Z)
    return (ez-esz)/(ez + esz)

def softmax(Z):
    """
    Computes softmax function for each row of numpy array Z.
    """
    A = np.exp(Z)
    A /= np.sum(A, axis=1, keepdims=True)
    return A

def forward_prop(X, Ws):
    """
    Forward propagates X through layers of neural nets to get the final outputs.
    
    Parameters
    ----------
    X : numpy array, shape (N, d+1)
        The matrix of input vectors (each row corresponds to an input vector); the first column of 
        this matrix is all ones (corresponding to x_0).
    Ws : list of numpy arrays
        The list of each layer's W; W of layer l will have the shape of (d^(l-1)+1, d^(l)) where 
        d^(l-1) is the number of neurons (not count the +1 neuron) of layer l-1, and 
        d^(l) is the number of neurons (not count the +1 neuron) of layer l.
    
    Returns
    -------
    A : numpy array, shape (N, K=10)
        The maxtrix of Neural Net's output vectors; each row is an output vector (containing each 
        class's probability given the corresponding input vector).
    """
    # TODO
    A = X;
    for i in range(len(Ws)):
        W = Ws[i];
        Z = A.dot(W)
        if i == len(Ws)-1:
            A = softmax(Z);
        else:
            A = tanh(Z);
            A = np.hstack((np.ones((A.shape[0],1)),A));
    return A;
      

In [4]:
def addOne(A):
    return np.hstack((np.ones((A.shape[0],1)),A));
def meanBinaryError(X, W, Y):
    #for debugging: def forward_prop(X, Ws):
    A = forward_prop(X, W);
    myY = np.nonzero(np.max(A, axis = 1, keepdims = True) == A)[1]
    error = 1 - np.sum(myY == Y) * 1.0 / Y.shape[0]
    return error
def train_neural_net(X_train, Y_train, X_val, Y_val, layer_sizes, learning_rate, mnb_size, max_patience, 
                     l2_reg_level):
    """
    Trains Neural Net on the dataset (X_train, Y_train).
    Cost function: Mean Negative Log Likelihood + L2 regularization.
    Optimization algorithm: Stochastic Gradient Descent (SGD) with early stopping.
    
    Parameters
    ----------
    X_train : numpy array, shape (N, d + 1)
        The matrix of input vectors (each row corresponds to an input vector) of the training set; 
        the first column of this matrix is all ones (corresponding to x_0).
    Y_train : numpy array, shape (N,)
        The vector of outputs of the training set.
    X_val : numpy array, shape (N_val, d + 1)
        The matrix of input vectors (each row corresponds to an input vector) of the validation set; 
        the first column of this matrix is all ones (corresponding to x_0).
    Y_val : numpy array, shape (N_val,)
        The vector of outputs of the validation set.  
    layer_sizes : list of ints
        The list of each layer' size (not count the +1 neurons).
        E.g. layer_sizes = [784, 30, 10] means: the 1st layer (input layer) has 784 neurons,
        the 2nd layer (hidden layer) has 30 neurons, the 3rd layer (output layer) has 10 neurons.
    learning_rate : float
        Learning rate of SGD.
    mnb_size : int
        Minibatch size of SGD.
    max_patience : int
        The parameter of early stopping. You'll have a `patience` variable with initial value equal to
        `max_patience`. During the training, you'll keep track of the best MBE (Mean Binary Error) 
        on the validation set; if the MBE on the validation set at the current epoch < the current 
        best one, you'll reset `patience` to `max_patience`; otherwise, `patience` -= 1. 
        When `patience` = 0, you'll terminate SGD.
    l2_reg_level : float
        The level (the coefficient) of L2 regularization.
    
    Returns
    -------
    (best_Ws, train_errs, val_errs) : tuple
        best_Ws : list of numpy arrays
            The list of each layer's W; W of layer l will have the shape of (d^(l-1)+1, d^(l)) where 
            d^(l-1) is the number of neurons (not count the +1 neuron) of layer l-1, and 
            d^(l) is the number of neurons (not count the +1 neuron) of layer l.
            It's the parameters having smallest MBE on the validation set.
        train_errs: list of floats
            List of MBEs on the training set after each epoch.
        val_errs: list of floats
            List of MBEs on the validation set after each epoch.
    
    Notes
    -----
    After each epoch, you need to print out: 
    - The MBE on the training set and validation set.
    - The value of `patience`.
    E.g., 'Epoch ..., training err ..., val err ..., patience ...'.
    
    After the training, you need to print out:
    - The best MBE on the validation set.
    - The corresponding epoch.
    - The corresponding MBE on the training set.
    E.g., "Best val err ..., at epoch ..., corresponding train err ...".
    """
    # Init weights
    np.random.seed(0) # Note: this will fix the randomization so that we'll get the same random numbers each run; 
                      # it make comparisons (e.g. between different values of `l2_reg_level`) more accurate. 
    Ws = [np.random.randn(layer_sizes[l]+1, layer_sizes[l+1]) / np.sqrt(layer_sizes[l]+1) for l in range(len(layer_sizes)-1)]
    # TODO
    bestMBEval = 111; #infinity ~ > 1
    corEpoch = -1;
    corMBEtrain = 111;
    train_errs = [];
    val_errs = [];
    bestWs = [];
    
    N = X_train.shape[0];
    one_hot_Y = np.eye(layer_sizes[-1]);
    rand_idxs = range(N);
    max_epoch = 1000000000; #infinity 
    epoch = -1;
    while epoch < max_epoch: #we can also use "While True:". However, we should use this condition instead to control when the loop terminates. 
        epoch += 1
        np.random.shuffle(rand_idxs);
        for start_idx in range(0,N, mnb_size):
            mnb_X = X_train[rand_idxs[start_idx:start_idx+mnb_size]];
            mnb_Y = one_hot_Y[Y_train[rand_idxs[start_idx:start_idx+mnb_size]]];
            #forward propagation
            As = [mnb_X];
            #Zs =[];
            A = mnb_X;
            for i in range(len(Ws)):
                W = Ws[i];
                Z = A.dot(W)
                if i == len(Ws)-1:
                    A = softmax(Z);
                else:
                    A = tanh(Z);
                    A = np.hstack((np.ones((A.shape[0],1)),A));
                    
                #Zs.append(Z); #for debuging
                As.append(A);
            #backward propagation
            delta = As[-1] - mnb_Y;
            grad = As[-2].T.dot(delta)*1.0/mnb_size + 2.0*l2_reg_level*Ws[-1]; #/mnb_size
            Ws[-1] -= learning_rate*grad;
            
            for it in range(2, len(layer_sizes)):
                delta = delta.dot(Ws[-it+1].T)*(1-As[-it]*As[-it])
                grad = (As[-it-1].T.dot(delta)*1.0/mnb_size)[:,1:] + 2.0*l2_reg_level*Ws[-it]; #/mnb_size
        
                Ws[-it] -= learning_rate*grad;
                delta = delta[:,1:];
        
        eTrain = meanBinaryError(X_train, Ws, Y_train);
        eVal = meanBinaryError(X_val, Ws, Y_val);
        train_errs.append(eTrain);
        val_errs.append(eVal);
        
        if (eVal < bestMBEval):
            bestMBEval = eVal;
            corMBEtrain = eTrain;
            corEpoch = epoch
            patience = max_patience
            bestWs = copy.deepcopy(Ws);
        else:
            patience = patience - 1;        
        print 'Epoch ', epoch, ', training err ', eTrain*100, '%, val err ', eVal*100, '%, patience ', patience, '\n'
        if patience == 0:
            break;
            
       
    print 'Best val err ', bestMBEval*100, '% at epoch ', corEpoch, ' corresponding train err ',corMBEtrain*100, '%';
    return (bestWs,train_errs,val_errs);

In [5]:
# TODO
X_train, Y_train, X_val, Y_val, X_test, Y_test = read_mnist('mnist.pkl.gz');
l2_reg = [0, 0.0001, 0.001];
bestWs = [];
train_errs = [];
val_errs = [];
#def train_neural_net(X_train, Y_train, X_val, Y_val, layer_sizes, learning_rate, mnb_size, max_patience, l2_reg_level):


### Thực nghiệm mạng nơ-ron 1 lớp ẩn
Thêm các kích thước lớp ẩn vào middle.

Ví dụ middle = [1,2,3]
- Huấn luyện 3 mạng có số nút của lớp ẩn lần lượt là 1, 2, 3.

In [6]:
middle = [150, 200, 300]
for mid in middle:
    bWs,tE,vE = train_neural_net(X_train, Y_train, X_val, Y_val, [784,mid,10], 0.1, 10, 20, 0.0001);
    bestWs.append((bWs, mid));
    train_errs.append(tE);
    val_errs.append(vE);

for WS, mid in bestWs:
    print  'Hidden layer ', mid,  '. Test error ', meanBinaryError(X_test, WS, Y_test)*100, '%';
    

Epoch  0 , training err  5.486 %, val err  5.26 %, patience  20 

Epoch  1 , training err  3.102 %, val err  3.54 %, patience  20 

Epoch  2 , training err  2.108 %, val err  2.92 %, patience  20 

Epoch  3 , training err  1.748 %, val err  2.65 %, patience  20 

Epoch  4 , training err  1.706 %, val err  2.69 %, patience  19 

Epoch  5 , training err  1.784 %, val err  2.99 %, patience  18 

Epoch  6 , training err  0.896 %, val err  2.08 %, patience  20 

Epoch  7 , training err  0.804 %, val err  2.33 %, patience  19 

Epoch  8 , training err  0.8 %, val err  2.38 %, patience  18 

Epoch  9 , training err  1.05 %, val err  2.6 %, patience  17 

Epoch  10 , training err  0.65 %, val err  2.21 %, patience  16 

Epoch  11 , training err  0.66 %, val err  2.14 %, patience  15 

Epoch  12 , training err  0.866 %, val err  2.41 %, patience  14 

Epoch  13 , training err  0.596 %, val err  2.12 %, patience  13 

Epoch  14 , training err  0.818 %, val err  2.45 %, patience  12 

Epoch  15 ,

### Thực nghiệm với mạng nơ-ron nhiều lớp. 
Thêm mô tả về mạng nơ-ron vào layer.
Ví dụ: layer = [[784,10], [784,30,10]]
- Huấn luyện 2 mạng nơ-ron với số lượng nút ở mỗi mạng là [784,10] và [784,30,10]

In [7]:
layer = [[784, 50, 30, 20, 10], [784, 50, 30, 20, 20, 10]]
for ll in layer:
    bWs,tE,vE = train_neural_net(X_train, Y_train, X_val, Y_val, ll, 0.1, 10, 20, 0.0001);
    bestWs.append((bWs, ll));
    train_errs.append(tE);
    val_errs.append(vE);

for WS, ll in bestWs:
    print ll 
    print  '. Test error ', meanBinaryError(X_test, WS, Y_test)*100, '%';

Epoch  0 , training err  6.446 %, val err  6.3 %, patience  20 

Epoch  1 , training err  3.788 %, val err  4.34 %, patience  20 

Epoch  2 , training err  3.428 %, val err  4.05 %, patience  20 

Epoch  3 , training err  2.758 %, val err  3.67 %, patience  20 

Epoch  4 , training err  2.396 %, val err  3.34 %, patience  20 

Epoch  5 , training err  2.572 %, val err  3.65 %, patience  19 

Epoch  6 , training err  2.42 %, val err  3.38 %, patience  18 

Epoch  7 , training err  2.112 %, val err  3.23 %, patience  20 

Epoch  8 , training err  2.048 %, val err  3.33 %, patience  19 

Epoch  9 , training err  1.602 %, val err  3.16 %, patience  20 

Epoch  10 , training err  1.69 %, val err  3.19 %, patience  19 

Epoch  11 , training err  2.03 %, val err  3.57 %, patience  18 

Epoch  12 , training err  2.194 %, val err  3.6 %, patience  17 

Epoch  13 , training err  1.674 %, val err  2.98 %, patience  20 

Epoch  14 , training err  1.726 %, val err  3.19 %, patience  19 

Epoch  15 